Download the dataset
-------------------------



In [0]:
!wget "http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz" -P "./data"
!tar -zxvf "/content/data/facades.tar.gz" -C "/content/data/" > "/content/data/output.log"

--2019-07-26 09:42:11--  http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.189.73
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30168306 (29M) [application/x-gzip]
Saving to: ‘./data/facades.tar.gz’

facades.tar.gz      100%[===================>]  28.77M  2.11MB/s    in 14s     

2019-07-26 09:42:25 (2.05 MB/s) - ‘./data/facades.tar.gz’ saved [30168306/30168306]



# Import PyTorch

In [0]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import os
import numpy as np
from torchvision.utils import save_image

# Some values

In [0]:
batch_size = 32
learning_rate = 0.002
beta1 = 0.5
beta2 = 0.999
z_dim = 100

cuda = True if torch.cuda.is_available else False

# Let's organize the dataset

In [0]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                        (0.5, 0.5, 0.5))
])

data_path = "./data/facades"

def create_dataset(path, mode):
  dataset = datasets.ImageFolder(path, transform=transform)
  index = dataset.class_to_idx[mode]
  
  n = 0
  for i in range(dataset.__len__()):
    if dataset.imgs[n][1] != index:
      del dataset.imgs[n]
      n -= 1
    n += 1
  return DataLoader(dataset, batch_size=batch_size, shuffle=True)
                                    

def make_dataset(path, index):
  dataset = datasets.ImageFolder(path, transform=transform)
#   index = dataset.class_to_idx[mode]
  
  n = 0
  for i in range(dataset.__len__()):
    if dataset.imgs[n][1] != index and dataset.imgs[n][1] != (index + 1):
      del dataset.imgs[n]
      n -= 1
    n += 1
  return DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_data = create_dataset(data_path, "train")
test_data = create_dataset(data_path, "test")

# Design the model

In [0]:
import torch.nn.functional as F

img_shape = (256, 256, 3)

class Generator(nn.Module):
    # initializers
    def __init__(self, d=64):
        super(Generator, self).__init__()
        # Unet encoder
        self.conv1 = nn.Conv2d(3, d, 4, 2, 1)
        self.conv2 = nn.Conv2d(d, d * 2, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d * 2)
        self.conv3 = nn.Conv2d(d * 2, d * 4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d * 4)
        self.conv4 = nn.Conv2d(d * 4, d * 8, 4, 2, 1)
        self.conv4_bn = nn.BatchNorm2d(d * 8)
        self.conv5 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.conv5_bn = nn.BatchNorm2d(d * 8)
        self.conv6 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.conv6_bn = nn.BatchNorm2d(d * 8)
        self.conv7 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.conv7_bn = nn.BatchNorm2d(d * 8)
        self.conv8 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        # self.conv8_bn = nn.BatchNorm2d(d * 8)

        # Unet decoder
        self.deconv1 = nn.ConvTranspose2d(d * 8, d * 8, 4, 2, 1)
        self.deconv1_bn = nn.BatchNorm2d(d * 8)
        self.deconv2 = nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d * 8)
        self.deconv3 = nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d * 8)
        self.deconv4 = nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.deconv4_bn = nn.BatchNorm2d(d * 8)
        self.deconv5 = nn.ConvTranspose2d(d * 8 * 2, d * 4, 4, 2, 1)
        self.deconv5_bn = nn.BatchNorm2d(d * 4)
        self.deconv6 = nn.ConvTranspose2d(d * 4 * 2, d * 2, 4, 2, 1)
        self.deconv6_bn = nn.BatchNorm2d(d * 2)
        self.deconv7 = nn.ConvTranspose2d(d * 2 * 2, d, 4, 2, 1)
        self.deconv7_bn = nn.BatchNorm2d(d)
        self.deconv8 = nn.ConvTranspose2d(d * 2, 3, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        e1 = self.conv1(input)
        e2 = self.conv2_bn(self.conv2(F.leaky_relu(e1, 0.2)))
        e3 = self.conv3_bn(self.conv3(F.leaky_relu(e2, 0.2)))
        e4 = self.conv4_bn(self.conv4(F.leaky_relu(e3, 0.2)))
        e5 = self.conv5_bn(self.conv5(F.leaky_relu(e4, 0.2)))
        e6 = self.conv6_bn(self.conv6(F.leaky_relu(e5, 0.2)))
        e7 = self.conv7_bn(self.conv7(F.leaky_relu(e6, 0.2)))
        e8 = self.conv8(F.leaky_relu(e7, 0.2))
        # e8 = self.conv8_bn(self.conv8(F.leaky_relu(e7, 0.2)))
        d1 = F.dropout(self.deconv1_bn(self.deconv1(F.relu(e8))), 0.5, training=True)
        d1 = torch.cat([d1, e7], 1)
        d2 = F.dropout(self.deconv2_bn(self.deconv2(F.relu(d1))), 0.5, training=True)
        d2 = torch.cat([d2, e6], 1)
        d3 = F.dropout(self.deconv3_bn(self.deconv3(F.relu(d2))), 0.5, training=True)
        d3 = torch.cat([d3, e5], 1)
        d4 = self.deconv4_bn(self.deconv4(F.relu(d3)))
        # d4 = F.dropout(self.deconv4_bn(self.deconv4(F.relu(d3))), 0.5)
        d4 = torch.cat([d4, e4], 1)
        d5 = self.deconv5_bn(self.deconv5(F.relu(d4)))
        d5 = torch.cat([d5, e3], 1)
        d6 = self.deconv6_bn(self.deconv6(F.relu(d5)))
        d6 = torch.cat([d6, e2], 1)
        d7 = self.deconv7_bn(self.deconv7(F.relu(d6)))
        d7 = torch.cat([d7, e1], 1)
        d8 = self.deconv8(F.relu(d7))
        o = torch.tanh(d8)

        return o

class Discriminator(nn.Module):
    # initializers
    def __init__(self, d=64):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(6, d, 4, 2, 1)
        self.conv2 = nn.Conv2d(d, d * 2, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d * 2)
        self.conv3 = nn.Conv2d(d * 2, d * 4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d * 4)
        self.conv4 = nn.Conv2d(d * 4, d * 8, 4, 1, 1)
        self.conv4_bn = nn.BatchNorm2d(d * 8)
        self.conv5 = nn.Conv2d(d * 8, 1, 4, 1, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = torch.sigmoid(self.conv5(x))

        return x

def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

# Define loss and optimizers

In [0]:
G = Generator()
D = Discriminator()
bce_loss = nn.BCELoss()
l1_loss = nn.L1Loss()

if cuda:
  G.cuda()
  D.cuda()
  bce_loss.cuda()
  l1_loss.cuda()

g_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=(beta1, beta2))
d_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas=(beta1, beta2))

# Time to train!

In [0]:
epochs = 200

os.makedirs("./images", exist_ok=True)

for epoch in range(epochs):
  
  for i, (data, _) in enumerate(train_data):
    
    if cuda:
      data = Variable(data.cuda())
      
    _y = Variable(data[:, :, :, :256].cuda())
    _x = Variable(data[:, :, :, 256:].cuda())
    
    # Train the Discriminator
    D.zero_grad()
    
    D_result = D(_x, _y)
    
    real = Variable(torch.ones(D_result.size()).cuda(), requires_grad=False)
    fake = Variable(torch.zeros(D_result.size()).cuda(), requires_grad=False)
    
    real_loss = bce_loss(D_result, real)
    G_result = G(_x)
    D_result = D(_x, G_result)
    fake_loss = bce_loss(D_result, fake)
    
    d_loss = (real_loss + fake_loss) / 2
    d_loss.backward()
    d_optimizer.step()
    
    # Train the Generator
    G.zero_grad()
    
    G_result = G(_x)
    D_result = D(_x, G_result)
    
    g_loss = bce_loss(D_result, real) + 100 * l1_loss(G_result, _y) 
    g_loss.backward()
    g_optimizer.step()
    
    
    batches_done = epoch * len(data) + i
    if batches_done % 10 == 0:
      print("Epoch %d, Batch %d" % (epoch, i))
      save_image(G_result.data[:25], "images/%d.png" % batches_done, 
                 nrow=5, 
                 normalize=True)
    
    

Epoch 0, Batch 0
Epoch 0, Batch 10
Epoch 1, Batch 8
Epoch 2, Batch 6
Epoch 3, Batch 4
Epoch 3, Batch 12
Epoch 4, Batch 2
Epoch 5, Batch 0
Epoch 5, Batch 10
Epoch 6, Batch 8
Epoch 7, Batch 6
Epoch 8, Batch 4
Epoch 8, Batch 12
Epoch 9, Batch 2
Epoch 10, Batch 0
Epoch 10, Batch 10
Epoch 11, Batch 8
Epoch 12, Batch 6
Epoch 13, Batch 4
Epoch 13, Batch 12
Epoch 14, Batch 2
Epoch 15, Batch 0
Epoch 15, Batch 10
Epoch 16, Batch 8
Epoch 17, Batch 6
Epoch 18, Batch 4
Epoch 18, Batch 12
Epoch 19, Batch 2
Epoch 20, Batch 0
Epoch 20, Batch 10
Epoch 21, Batch 8
Epoch 22, Batch 6
Epoch 23, Batch 4
Epoch 23, Batch 12
Epoch 24, Batch 2
Epoch 25, Batch 0
Epoch 25, Batch 10
Epoch 26, Batch 8
Epoch 27, Batch 6
Epoch 28, Batch 4
Epoch 28, Batch 12
Epoch 29, Batch 2
Epoch 30, Batch 0
Epoch 30, Batch 10
Epoch 31, Batch 8
Epoch 32, Batch 6
Epoch 33, Batch 4
Epoch 33, Batch 12
Epoch 34, Batch 2
Epoch 35, Batch 0
Epoch 35, Batch 10
Epoch 36, Batch 8
Epoch 37, Batch 6
Epoch 38, Batch 4
Epoch 38, Batch 12
Epoch 39